# Text generation with LSTM Recurrent Neural Networks

By Alex Gascón Bononad - alexgascon.93@gmail.com

## 0. Introduction

### 0.1. Introduction to the Notebook
In this notebook we'll follow the following tutorial: http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ [1]

As the title states, we're going to use Python and Keras to create a LSTM Recurrent Neural Network to use for text generation. We'll start by following what the tutorial specifies and then we'll try some variations that let us experimentate and get different results


### 0.2. Introduction to the concepts

There are several concepts that will be useful to know during this tutorial

#### Text generation
Text Generation or Natural language generation (NLG) is the natural language processing task of generating natural language from a machine representation system. It could be said an NLG system is like a translator that converts data into a natural language representation. 

NLG may be viewed as the opposite of natural language understanding: whereas in natural language understanding the system needs to disambiguate the input sentence to produce the machine representation language, in NLG the system needs to make decisions about how to put a concept into words.

_More info:_
https://en.wikipedia.org/wiki/Natural_language_generation [2]

#### LSTM Recurrent Neural Networks
LSTM is the acronym for Long short-term memory, a type of Recurrent Neural Networks capable of learning long-term dependencies. They are explicitly designed to avoid the long-term dependency problem, so they don't struggle at all to remember information for long periods of time.

_More info:_ <br />
http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf [3] <br />
http://colah.github.io/posts/2015-08-Understanding-LSTMs/ [4]



## 1. Description of the problem
As we've previously explained, the purpose of this notebook is to generate text using a Neural Network. 

In order to do this, we need a big amount of information to train the NN, and in this case, we've chosen the book _Alice's Adventures in Wonderland_, by Lewis Carroll. It's no longer protected with copyright, and we can find it on [Project Gutenberg](https://www.gutenberg.org/). Specifically, we're going to use the text version of the book.

First of all, we'll remove the header and footer of the book, as it contains generic info about Project Gutenberg that we don't want to include in the training set. This doesn't mean that it isn't important (in fact, as we're using information they've hosted and distributed, the least we can do for the project is to read these information), but that we don't want to analyse it as it may greatly affect our output. You can find the edited .txt file in the same directory as this notebook, with the name "Alice's adventures in Wonderland.txt"


## 2. Develop a Small LSTM Recurrent Neural Network

Now let's start the coding! The first steps will be to load the libraries that we'll need during this notebook and to read the text file. Besides, we'll convert it to lowercase in order to simplify the analysis.

In [1]:
# Setting theano tu run in GPU
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [2]:
# Reading the book and converting to lowercase
filename = "Alice's adventures in Wonderland.txt"
book = open(filename).read()
book = book.lower()

Now, we'll prepare the data to be modelled by the Neural Network. In order to do this we'll map each character to their corresponding ASCII integer.

In [3]:
# Creating a sorted list where every char in the book appears just once
chars = sorted(list(set(book)))
# Getting a dictionary that representa the mapping of every char (key) to their corresponding int (value)
char_to_int = dict((c, i) for i, c in enumerate(chars))

Up to this point, we have the following data:

In [4]:
print("Total characters in the book: ", len(book))
print("Unique characters in the book: ", len(chars))

('Total characters in the book: ', 144588)
('Unique characters in the book: ', 48)


In this step we'll start to define the training data for our LSTM RNN. In order to follow the original tutorial, what we'll do is to split the whole book into strings of 100 characters, and consider the following one as the expected output. 

We will slide this 100-character window along the whole book one chatacter at a time, so each char will be learned 100 times (with the exception of the first 100 characters). 

As a simpler way of explaining this: if instead of a window of length 100 we'll use one of length 5, the following ones will be some of the training inputs:

CHAPT --> E <br/>
HAPTE --> R <br/>
wonde --> r <br/>
onder --> l <br/>
nderl --> a

In [5]:
# Preparing the dataset of input-output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

# Iterating over the book
for i in range(0, len(book) - seq_length):
    sequence_in = book[i : i + seq_length]
    sequence_out = book[i + seq_length]
    
    # Converting each char to its corresponding int
    sequence_in_int = [char_to_int[char] for char in sequence_in]
    sequence_out_int = char_to_int[sequence_out]
    
    # Appending the result to the current data 
    dataX.append(sequence_in_int)
    dataY.append(sequence_out_int)
    
print("Total patterns: ", len(dataX))

('Total patterns: ', 144488)


As we can see, the total amount of input patterns is exactly the total amount of characters in the text - 100. This, of course, is not a coincidence: we have a pattern to predict each of the characters, except for the first 100 (the value of our seq_length) because they don't have enough preceding characters to be predicted. 

The next step will be to transform the training data to make it suitable for its use in Keras.

We'll do this in three steps: the first one is to transform the list of input sequences into the form _[samples, time steps, features]_ expected by an LSTM RNN. 
- **Samples**: the amount of input elements to use during our training
- **Time steps**: indicates how long will a single input be kept in our network (i.e. how many iterations will occur before it leaves the network)
- **Features**: How many features does the network predict each iteration

In [6]:
# Reshaping X to be [samples, time_steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, len(sequence_out)))

The next step will be to normalize the outputs between 0 and 1, in order to make the patterns easier to learn by the LSTM network, that by default uses the sigmoid activation function.

In [7]:
# Normalizing between 0 and 1
X = X / float(len(chars))

Finally, we need to convert the output patterns into a one hot enconding. This encoding is represented with a group of bits only one single can have a value of 1, with the other ones being 0. 

Let's see some examples of binary to one-hot conversions: <br />
_000 --> 00000001 <br />
001 --> 00000010 <br />
010 --> 00000100 <br />
011 --> 00001000 <br />
100 --> 00010000 <br />
101 --> 00100000 <br />
110 --> 01000000 <br />
111 --> 10000000_

In our case, the one-hot encoding will be represented by an array formed by 46 items (the amount of unique characters in our text) in which only one of them will be a 1. For example, the value of "n" (integer value 31) will be the following one:

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0. 0.  0.  0.  0.  0.  0.  0.  0.]

In [8]:
# One hot encode the output variable
y = np_utils.to_categorical(dataY)

With this, we've finished the data preparation and it's time to start defining our LSTM model. We'll use the same parameters from the tutorial:

- **Hidden layers:** 1
- **Units per hidden layer:** 256
- **Dropout probability:** 20
- **Output layer:** Dense layer
- **Output activation function:** Softmax.

In case that any of the terms was unknown to you (like happened to me when I first started with NN), you can find more information about them in the following lines and links:

- *Dropout probability:* Represents the probability of a neuron to be ignored. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.
You can imagine that if neurons are randomly dropped out of the network during training, that other neurons will have to step in and handle the representation required to make predictions for the missing neurons. The effect is that the network becomes less sensitive to the specific weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overfit the training data. *More info:* http://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/ [5]

- *Dense layer:* A fully-connected layer, i.e. layer where every neuron is connected to each neuron in the next layer.

- *Softmax function*: [Softmax function in Wikipedia](https://en.wikipedia.org/wiki/Softmax_function) [6]
![Graph of the Softmax function](https://qph.ec.quoracdn.net/main-qimg-fda2f008df90ed5d7b6aff89b881e1ac)

Finally, for the compilation we're going to optimize the log loss (cross entropy), here using the ADAM optimization algorithm for speed.

In [9]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))       # Adding the hidden layer
model.add(Dropout(0.2))                                          # Indicating the Dropout probability
model.add(Dense(y.shape[1], activation='softmax'))               # Adding the output layer
model.compile(loss='categorical_crossentropy', optimizer='adam') # Setting the compilation

There is no test dataset. We are modeling the entire training dataset to learn the probability of each character in a sequence.

We are not interested in the most accurate (classification accuracy) model of the training dataset. This would be a model that predicts each character in the training dataset perfectly. Instead we are interested in a generalization of the dataset that minimizes the chosen loss function. We are seeking a balance between generalization and overfitting but short of memorization.

The network is slow to train (about 300 seconds per epoch on an Nvidia K520 GPU). Because of the slowness and because of our optimization requirements, we will use model checkpointing to record all of the network weights to file each time an improvement in loss is observed at the end of the epoch. We will use the best set of weights (lowest loss) to instantiate our generative model in the next section.

In [10]:
# Define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

We can now fit our model to the data. Here we use a modest number of 20 epochs and a large batch size of 128 patterns.

In [11]:
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list, verbose=1)

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/axelp/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.13-64/tmpxkg7jn/da6ac6908198a9d83223e8a2be6b76b9.lib and object C:/Users/axelp/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.13-64/tmpxkg7jn/da6ac6908198a9d83223e8a2be6b76b9.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/axelp/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.13-64/tmpnebnj_/3822e09213127a9018d8f73e171845df.lib and object C:/Users/axelp/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.13-64/tmpnebnj_/3822e09213127a9018d8f73e171845df.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/axelp/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.13-64/tmpvo

Epoch 1/20
144488/144488 [==============================] - 459s - loss: 2.9535   
Epoch 2/20
144488/144488 [==============================] - 449s - loss: 2.7650   
Epoch 3/20
144488/144488 [==============================] - 451s - loss: 2.6741   
Epoch 4/20
144488/144488 [==============================] - 447s - loss: 2.6040   
Epoch 5/20
144488/144488 [==============================] - 459s - loss: 2.5451   
Epoch 6/20
144488/144488 [==============================] - 453s - loss: 2.4933   
Epoch 7/20
144488/144488 [==============================] - 444s - loss: 2.4464   
Epoch 8/20
144488/144488 [==============================] - 452s - loss: 2.4044   
Epoch 9/20
144488/144488 [==============================] - 466s - loss: 2.3602   
Epoch 10/20
144488/144488 [==============================] - 456s - loss: 2.3203   
Epoch 11/20
144488/144488 [==============================] - 452s - loss: 2.2830   
Epoch 12/20
144488/144488 [==============================] - 695s - loss: 2.2457   
E